In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import *
from konlpy.tag import Okt
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
pd.set_option('display.unicode.east_asian_width', True)

In [2]:
df_test = pd.read_csv('./crawling/news_test_dataconcat.csv', index_col = 0)
print(df_test.head())

# dict_data = {'title':['김종민 "올해 수십조 이상 투입해야 할 수도"', '[서환] 글로벌 달러 강세에 1,120원대로 상승…2.30원↑', 'ANZ "키위달러 예상외 약세…강달러 영향"', '中 가전제품 제조사 메이디, 4천억 원 투자해 의료기기 사업 진입', '네이버, 라이브커머스 방송인 "라이브스타" 정식 계약', '2·4 부동산 대책 수혜주는…"건설주 단기 센티먼트는 약화"', '건설사들, 수도권 중심으로 분양 여건 개선 기대' ], 
#              'category':['Policy', 'Bond', 'Global_Economy', 'China_Economy', 'IB', 'Stock', 'Real_Estate' ]}
# df_test = pd.DataFrame(dict_data)

 
df_test

                                                      title category
0       김종민  올해 수십조 이상 투입해야 할 수도             Policy
1     금감원  비대면으로 설맞이 전통시장 물품 구입            Policy
2   은행권  자산 증여받는  apos MZ세대 apos  잡아라  ......   Policy
3        김용범 기재차관  양계 산업기반 회복 지원             Policy
4         기보  인천 기술중소기업 1천억원 우대보증            Policy


,title,category
0,김종민 올해 수십조 이상 투입해야 할 수도,Policy
1,금감원 비대면으로 설맞이 전통시장 물품 구입,Policy
2,은행권 자산 증여받는 apos MZ세대 apos 잡아라 ...,Policy
3,김용범 기재차관 양계 산업기반 회복 지원,Policy
4,기보 인천 기술중소기업 1천억원 우대보증,Policy
...,...,...
6983,홍대 상권도 코로나19 직격탄 카페 월매출 작년말보다 708만원 ...,Real_Estate
6984,국토硏 거래 늘어난다고 아파트값이 오르진 않아 ...,Real_Estate
6985,현대ENG 대만서 4천억 규모 발전소 증설공사 수주 ...,Real_Estate
6986,5년만에 재건축 수주전 뛰어든 삼성물산 계열사 총결집 협업 ...,Real_Estate


In [3]:
stopwords = pd.read_csv('../datasets/stopwords.csv')

In [4]:
model = load_model('./news_finance.h5')

In [5]:
with open('./news_token.pickle', 'rb') as f:
    token = pickle.load(f)

In [6]:
print(df_test.iloc[3])

title        김용범 기재차관  양계 산업기반 회복 지원   
category                                          Policy
Name: 3, dtype: object


In [7]:
test_title = df_test.iloc[3]['title']
print(test_title)

okt = Okt()
test_title = okt.morphs(test_title)
result = []

for i in range(len(test_title)):
    if test_title[i] not in list(stopwords['stopword']):
        result.append(test_title[i])
test_title = result
test_title = ' '.join(test_title)
print(test_title)

 김용범 기재차관  양계 산업기반 회복 지원   
김용범 기재 차관 양계 산업 기반 회복 지원


In [8]:
# padding

test_title = token.texts_to_sequences([test_title])
test_title = pad_sequences(test_title, 17)
print(test_title)

[[   0    0    0    0    0    0    0    0    0    0  525  574  372  231
  1011  108   89]]


In [18]:
label = ['Policy', 'Bond', 'Global_Economy', 'China_Economy', 'IB', 'Stock', 'Real_Estate']
label.sort()
print(label)

['Bond', 'China_Economy', 'Global_Economy', 'IB', 'Policy', 'Real_Estate', 'Stock']


In [19]:
predict = model.predict(test_title)
print(predict)
print(label[np.argmax(predict)]) 

[[1.9086420e-04 4.4403521e-07 6.1469247e-05 2.4799777e-03 9.9634391e-01
  1.1985544e-06 9.2216849e-04]]
Policy


In [20]:
# 하나씩 말고 전체 파일로 test하기
# 데이터프레임 객체에서 인덱싱 -> 리스트화

stopwords = list(pd.read_csv('../datasets/stopwords.csv')['stopword'])
test_titles = []
predicts = []

for title in df_test['title']:
    # 형태소
    title = okt.morphs(title)
    result = []
    
    # 불용어, 한 글자 처리
    for word in title:
        if (word not in stopwords) and len(word) > 1:
            result.append(word)
    title = result
    title = ' '.join(title)
    
    # 토크나이징 (모델 기준 패딩)
    title = token.texts_to_sequences([title])
    title = pad_sequences(title, 17)
    
    # 예측
    predict = model.predict(title)
    predicts.append(label[np.argmax(predict)])

In [21]:
# 예측 결과를 새로운 칼럼으로 추가
df_test['predict'] = predicts
print(df_test.head())

                                                      title category  \
0       김종민  올해 수십조 이상 투입해야 할 수도             Policy   
1     금감원  비대면으로 설맞이 전통시장 물품 구입            Policy   
2   은행권  자산 증여받는  apos MZ세대 apos  잡아라  ......   Policy   
3        김용범 기재차관  양계 산업기반 회복 지원             Policy   
4         기보  인천 기술중소기업 1천억원 우대보증            Policy   

       predict OX  
0           IB  X  
1       Policy  X  
2       Policy  X  
3       Policy  X  
4  Real_Estate  X  


In [22]:
# 맞으면 O, 틀리면 X 표시하는 칼럼 추가

df_test['OX'] = 0
for i in range(len(df_test.predict)):
    if df_test.category[i] == df_test.predict[i]:
        df_test.OX[i] = 'O'
    else:
        df_test.OX[i] = 'X'
print(df_test)

C:\Users\698\anaconda3\envs\AI_dev_env\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\698\anaconda3\envs\AI_dev_env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


                                                                      title  \
0          김종민  올해 수십조 이상 투입해야 할 수도                          
1        금감원  비대면으로 설맞이 전통시장 물품 구입                         
2      은행권  자산 증여받는  apos MZ세대 apos  잡아라  ......                
3           김용범 기재차관  양계 산업기반 회복 지원                          
4            기보  인천 기술중소기업 1천억원 우대보증                         
...                                                 ...                       
6983   홍대 상권도 코로나19 직격탄 카페 월매출 작년말보다 708만원    ......   
6984   국토硏  거래 늘어난다고 아파트값이 오르진 않아   ......                
6985   현대ENG  대만서 4천억 규모 발전소 증설공사 수주  ......                
6986   5년만에 재건축 수주전 뛰어든 삼성물산  계열사 총결집 협업   ......     
6987   LH  신혼희망타운 올해 8월부터 공급 12월 71  물량 집중  ......          

         category      predict OX  
0          Policy           IB  X  
1          Policy       Policy  O  
2          Policy       Policy  O  
3          Policy       Policy  O  
4          Policy  Real_Estate  X  
...           ...     

In [23]:
# 얼마나 맞히고 틀렸는지
df_test.OX.value_counts()

# O만 세기
df_test.OX.value_counts()['O']

# 정답률
df_test.OX.value_counts()['O']/len(df_test.OX)

0.8986834573554665

In [24]:
df_test.OX.value_counts(normalize=True)['O'] * 100

89.86834573554665